## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-08-18-51-00 +0000,wsj,Trump Preparing IPO for Fannie Mae and Freddie...,https://www.wsj.com/finance/regulation/trump-a...
1,2025-08-08-18-46-52 +0000,nyt,Trump to Meet With Leaders of Armenia and Azer...,https://www.nytimes.com/2025/08/08/us/politics...
2,2025-08-08-18-46-42 +0000,startribune,Minnesota hasn’t had a GOP governor in almost ...,https://www.startribune.com/minnesota-hasnt-ha...
3,2025-08-08-18-45-00 +0000,wsj,Stock Market Today: Gold Pares Gains; Dow Rises,https://www.wsj.com/livecoverage/stock-market-...
4,2025-08-08-18-44-51 +0000,nyt,Trump Wants U.C.L.A. to Pay $1 Billion to Rest...,https://www.nytimes.com/2025/08/08/us/trump-wa...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2322/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,75
174,gaza,27
204,new,17
171,israel,16
172,military,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
67,2025-08-08-16-13-00 +0000,wsj,President Trump has directed the Pentagon to b...,https://www.wsj.com/politics/national-security...,165
258,2025-08-07-21-50-00 +0000,wsj,President Trump said he would nominate Stephen...,https://www.wsj.com/politics/policy/stephen-mi...,137
72,2025-08-08-16-06-14 +0000,nypost,Trump secretly authorized military force agai...,https://nypost.com/2025/08/08/us-news/trump-se...,125
77,2025-08-08-15-42-00 +0000,wsj,Trump’s Fed nominee would add a voice inside t...,https://www.wsj.com/economy/central-banking/st...,117
35,2025-08-08-17-25-18 +0000,nypost,Donald Trump Jr. mocked for posting photoshopp...,https://nypost.com/2025/08/08/us-news/trump-jr...,111


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
67,165,2025-08-08-16-13-00 +0000,wsj,President Trump has directed the Pentagon to b...,https://www.wsj.com/politics/national-security...
166,83,2025-08-08-09-35-51 +0000,bbc,Israel approves plan to take control of Gaza C...,https://www.bbc.com/news/articles/c8ryekj1m1do...
242,53,2025-08-07-22-56-06 +0000,nyt,California Supreme Court Requires New Review o...,https://www.nytimes.com/2025/08/07/business/en...
258,53,2025-08-07-21-50-00 +0000,wsj,President Trump said he would nominate Stephen...,https://www.wsj.com/politics/policy/stephen-mi...
245,42,2025-08-07-22-40-47 +0000,nypost,Disney settles Gina Carano’s lawsuit after fir...,https://nypost.com/2025/08/07/media/disney-set...
50,40,2025-08-08-16-47-10 +0000,nypost,JD Vance goes fishing with top UK official Dav...,https://nypost.com/2025/08/08/us-news/jd-vance...
225,39,2025-08-08-00-24-53 +0000,bbc,"Dozens of migrants detained under 'one in, one...",https://www.bbc.com/news/articles/ce35v0zyzvlo...
248,36,2025-08-07-22-24-21 +0000,nypost,Air Force denies early retirement pay to some ...,https://nypost.com/2025/08/07/us-news/air-forc...
77,36,2025-08-08-15-42-00 +0000,wsj,Trump’s Fed nominee would add a voice inside t...,https://www.wsj.com/economy/central-banking/st...
249,35,2025-08-07-22-20-42 +0000,nypost,Trusted Biden aide testifies ex-prez skipped S...,https://nypost.com/2025/08/07/us-news/trusted-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
